<a target="_blank" href="https://github.com/sakarimov/Basic-ML-Model/blob/main/JanKenPon%20CNN%20Image%20Recognizer.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Student Name** : Sulthan A. Karimov
# **Username** : sulthankarimov
# **Email** : sulthankarimov@gmail.com

In [1]:
!pip install split-folders[full] matplotlib jupyter_http_over_ws

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

In [3]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [4]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

IN_COLAB

False

In [14]:
# sometime i run this notebook on my laptop for testing purpose
local_dir = (
    'datasets/images/' if IN_COLAB == False
    else '/tmp/'
)

dataset_name = 'rockpaperscissors'
local_data = local_dir + dataset_name
local_zip = local_data + '.zip'
print(local_zip)

datasets/images/rockpaperscissors.zip


In [18]:
!test -f $local_zip || wget --no-check-certificate \
 https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
 -o $local_zip

In [19]:
import zipfile, os, shutil, splitfolders, re

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall(local_dir)
zip_ref.close()

shutil.rmtree(local_data + '/rps-cv-images')
os.remove(local_data + '/README_rpc-cv-images.txt')
print(os.listdir(local_data))

base_dir = local_dir + '/rps'
if os.path.exists(base_dir) == True:
  shutil.rmtree(base_dir)

splitfolders.ratio(local_data, base_dir, ratio=(.6,.4))
train_dir = os.path.join(base_dir, 'train')
print()
print('amount of training sample : ', sum(len(files) for _, _, files in os.walk(re.escape(base_dir) + r'/train')))
validation_dir = os.path.join(base_dir, 'val')
print('amount of validation sample : ', sum(len(files) for _, _, files in os.walk(re.escape(base_dir) + r'/val')))

['paper', 'rock', 'scissors']


Copying files: 2188 files [00:00, 5503.51 files/s]


amount of training sample :  1312
amount of validation sample :  876


In [20]:
os.listdir(train_dir)

['paper', 'rock', 'scissors']

In [21]:
os.listdir(validation_dir)

['paper', 'rock', 'scissors']

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
    rescale = 1./255)

In [23]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 4,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    batch_size = 4,
    class_mode = 'categorical')

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


In [24]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2
transfer = tf.keras.models.Sequential([
    ResNet152V2(
        weights = "imagenet", 
        include_top = False, 
        input_tensor = tf.keras.layers.Input(shape = (150,150,3))),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])

In [29]:
transfer.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.optimizers.Adam(),
              metrics = ['accuracy'])

In [30]:
early = tf.keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True
    )

In [31]:
transfer.fit(
    train_generator,
    epochs = 40,
    validation_data = validation_generator,
    validation_steps = 5,
    verbose = 2,
    callbacks = [early]
)

Epoch 1/40


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1715744121.767826     760 service.cc:145] XLA service 0x781b70002780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715744121.767857     760 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 960M, Compute Capability 5.0
2024-05-15 03:35:27.431713: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at variable_ops.cc:90 : FAILED_PRECONDITION: Read variable failure sequential/dense_1_1/bias/822. It could mean the variable is uninitialized or the variable is on another device 
2024-05-15 03:35:27.520194: W tensorflow/core/framewor

FailedPreconditionError: Graph execution error:

Detected at node sequential_1/dense_1_2/add/ReadVariableOp defined at (most recent call last):
<stack traces unavailable>
Read variable failure sequential/dense_1_1/bias/822. It could mean the variable is uninitialized or the variable is on another device 
	 [[{{node sequential_1/dense_1_2/add/ReadVariableOp}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_112848[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]] [Op:__inference_one_step_on_iterator_116565]